In [668]:
import pandas as pd
import re
import sqlalchemy
import unicodedata
import difflib
import xlwings as xw
from datetime import datetime
from googletrans import Translator
from os import listdir
from os.path import isfile, join

mypath = 'C:\\Users\\anicieja\Documents\\OBJ\\PL_automation\\source_new\\stawki'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [669]:
onlyfiles = [file for file in onlyfiles if not '~' in file]
onlyfiles


['ZESTAWIENIE WYNAGRODZEŃ_102018.xlsx']

In [670]:
def get_protected_sheet_data(PATH):
    wb = xw.Book(PATH)

    salaries = []
    for sheet in wb.sheets:
        print(sheet)

        data_from_sheet = sheet.range('A1').expand().options(pd.DataFrame, index=False, header=True).value

        # data_from_sheet['Etat'] = data_from_sheet['Etat']

        data_from_sheet.rename(columns={"Tytul": "Stanowisko",
                                        "Kwota": "Stawka",
                                        "kwota": "Stawka",
                                        "Wartość": "Stawka",
                                        "wartość": "Stawka",
                                        "Nazwisko i imię": "Pracownik",
                                        "Nazwisko": "Pracownik",
                                        "ETAT": 'Etat'}, inplace=True)

        data_from_sheet['FinanceSheet'] = sheet.name
        
        if "B2B" in sheet.name:
            data_from_sheet['Etat'] = "B2B"
            data_from_sheet['Typ umowy'] = 'B2B'
        
        if 'UOP' in sheet.name:
            data_from_sheet['Pracownik'] = data_from_sheet['Pracownik'] + ' ' + data_from_sheet['Imie']
        
        date_tmp = wb.name[26:30] + '-' + wb.name[24:26] + '-' + '01'
        print(date_tmp)
        data_from_sheet['Date'] = date_tmp
        data_from_sheet['Date'] = data_from_sheet['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
        
        # full_cost_columns = data_from_sheet.columns.values
        # data_from_sheet['EnovaEmployerCost'] = data_from_sheet.apply(lambda x: get_full_cost(x, full_cost_columns), axis=1)

        salaries.append(data_from_sheet[['Kod', 'Pracownik', 'Stanowisko', 'Etat',
                                         'Typ umowy', 'FinanceSheet', 'Stawka', 'Date']])

    return salaries

def etat_to_number(x):
    g = re.match('([0-9])/([0-9])', str(x).replace("'", ""))
    if g is None:
        return g
    g = g.groups()
    if len(g) == 1:
        return g[0]
    return float(g[0])/float(g[1])

def get_name(x):
    x = unicodedata.normalize('NFD', x.lower()).replace('ł', 'l').encode('ascii', 'ignore').decode()
    if '(' in x:
        full_name = re.match('(.{1,}) (.{1,}) \(.{1,}\)', x).groups()
    else:
        full_name = re.match('(.{1,}) (.{1,})', x).groups()
    return full_name[1].title() + ' ' + full_name[0].title()

In [671]:
salaries_array = []
for name in onlyfiles:
    print(name)
    salaries_array.append(pd.concat(get_protected_sheet_data(mypath + '\\' + name)))
    

ZESTAWIENIE WYNAGRODZEŃ_102018.xlsx


<Sheet [ZESTAWIENIE WYNAGRODZEŃ_102018.xlsx]UOP_OBSS>


2018-10-01
<Sheet [ZESTAWIENIE WYNAGRODZEŃ_102018.xlsx]B2B_OBSS>
2018-10-01


<Sheet [ZESTAWIENIE WYNAGRODZEŃ_102018.xlsx]UOP_OITS>
2018-10-01
<Sheet [ZESTAWIENIE WYNAGRODZEŃ_102018.xlsx]B2B_OITS>


2018-10-01


In [672]:
salaries = pd.concat(salaries_array)

salaries = salaries.rename(columns={
    'Kod': 'FinanceCode',
    'Pracownik': 'FinanceName',
    'Stanowisko': 'FinancePosition',
    'Etat': 'PartTime',
    'Stawka': 'ContractValue',
    'Typ umowy': 'ContractType',
})

salaries['PartTime'] = salaries['PartTime'].apply(etat_to_number) 
salaries = salaries.drop_duplicates()
# tmp = 4 - len(salaries['FinanceCode'])
# for x in range(tmp):
#     salaries['FinanceCode'] = "0" + salaries['FinanceCode']
# 
# salaries['FinanceCode'] = str(salaries['FinanceCode'])
# print(salaries['FinanceCode'])
salaries['CorrectedFinanceName'] = salaries['FinanceName'].apply(get_name)
salaries['CorrectedFinanceName'] = salaries['CorrectedFinanceName'].str.upper()
salaries['PartTime'] = salaries['PartTime'].fillna(1)
# salaries['CorrectedFinanceName'] = "(" + salaries['FinanceCode'] + ") " + salaries['FinanceName'].str.upper()

In [673]:
salaries.head()

,FinanceCode,FinanceName,FinancePosition,PartTime,ContractType,FinanceSheet,ContractValue,Date,CorrectedFinanceName
0,74,ADAMSKA URSZULA,SENIOR PROJECT MANAGER,0.8,Na czas nieokreślony,UOP_OBSS,10000.0,2018-10-01,URSZULA ADAMSKA
1,41,ADAMSKA-WOŹNY ALEKSANDRA,Assistant Guild Master,1.0,Na czas nieokreślony,UOP_OBSS,8000.0,2018-10-01,ALEKSANDRA ADAMSKA-WOZNY
2,821,ANDRUSIECZKO ANDRZEJ,Junior Quality Engineer/ Młodszy Tester Oprogr...,1.0,Na czas nieokreślony,UOP_OBSS,5000.0,2018-10-01,ANDRZEJ ANDRUSIECZKO
3,157,ANDRZEJEWSKI ADRIAN,Chapter Leader Service Delivery Manager,1.0,Na czas nieokreślony,UOP_OBSS,12000.0,2018-10-01,ADRIAN ANDRZEJEWSKI
4,802,ANTONETS IULIIA,ANALITYK BIZNESOWY,1.0,Na czas nieokreślony,UOP_OBSS,12500.0,2018-10-01,IULIIA ANTONETS


In [674]:
FIN_SERVER = "OBJPLDMFIN1"
FIN_DATABASE = "Cost"
FIN_SCHEMA = "dbo"
FIN_DB_CONNECTION = 'mssql+pyodbc://@' + FIN_SERVER + '/' + FIN_DATABASE + '?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes'
FIN_ENGINE = sqlalchemy.create_engine(FIN_DB_CONNECTION, pool_size=20, max_overflow=100)

In [675]:
salaries.reset_index()
salaries.to_sql('Sal_con', FIN_ENGINE.connect(), if_exists='replace')

In [676]:
DM_SERVER = "OBJPLDMSQL"
DM_DATABASE = "Structure"
DM_SCHEMA = "dbo"
DM_DB_CONNECTION = 'mssql+pyodbc://@' + DM_SERVER + '/' + DM_DATABASE + '?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes'
DM_ENGINE = sqlalchemy.create_engine(DM_DB_CONNECTION, pool_size=20, max_overflow=100)

In [677]:
employees = pd.read_sql('''
select * FROM [Structure].[dbo].[Structure_Monthly] where Date >= '{0}' and Date <= '{1}'
'''.format(salaries['Date'].min(), salaries['Date'].max()), DM_ENGINE.connect())

In [678]:
employees['Level'] = employees['Level'].replace({'': 'Regular'})

In [679]:
names_changes = pd.read_sql('''
SELECT distinct concat([First_Name], ' ', [Surname]) as NameFrom
      ,concat([First_Name], ' ', [Previous_Surname]) as NameTo
FROM [Translations].[dbo].[Maiden_Name]

UNION ALL

SELECT distinct concat([First_Name], ' ', [Previous_Surname]) as NameFrom
      ,concat([First_Name], ' ', [Surname]) as NameTo
FROM [Translations].[dbo].[Maiden_Name]
''', DM_ENGINE.connect())

In [680]:
translator = Translator()
positions = salaries['FinancePosition'].unique()
translations = []
for x in positions:
    #print(x)
    try:
        translations.append(translator.translate(x.replace('/', ''), dest='en').text)
    except Exception as e:
        translations.append(x)
positions_translations = pd.DataFrame({'FinancePosition': positions, 'FinanceEnglishPosition': translations})
positions_translations.head()

,FinancePosition,FinanceEnglishPosition
0,SENIOR PROJECT MANAGER,SENIOR PROJECT MANAGER
1,Assistant Guild Master,Assistant Guild Master
2,Junior Quality Engineer/ Młodszy Tester Oprogr...,Junior Quality Engineer/ Młodszy Tester Oprogr...
3,Chapter Leader Service Delivery Manager,Chapter Leader Service Delivery Manager
4,ANALITYK BIZNESOWY,ANALITYK BIZNESOWY


In [681]:
salaries = salaries.merge(positions_translations, how='left')
salaries.shape

(656, 10)

In [682]:
def get_matched_record(record, name_column='CorrectedFinanceName', position_column='FinanceEnglishPosition', prefix=''):
    record = record.iloc[0]
    x = record[name_column]
    #print(x)

    ratios = pd.DataFrame({'Name': employees['Name'].unique(),
                           'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in employees['Name'].unique()]
                           })

    max_ratio = ratios['Ratio'].max() - 0.1
    if max_ratio < 0.85:
        print(x)
        print(ratios['Ratio'].max())

        ratios_maiden = pd.DataFrame({'Name': names_changes['NameFrom'].tolist(),
                                      'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in names_changes['NameFrom'].tolist()]
                                      })
        max_ratio_maiden = ratios_maiden['Ratio'].max()
        matched_name_maiden = ratios_maiden[ratios_maiden['Ratio'] >= max_ratio_maiden]
        if matched_name_maiden.shape[0] >= 1 and (max_ratio_maiden - 0.1) > max_ratio:
            x = names_changes['NameTo'][names_changes['NameFrom'] == matched_name_maiden['Name'].iloc[0]].iloc[0]

            ratios = pd.DataFrame({'Name': employees['Name'].unique(),
                                   'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in employees['Name'].unique()]
                                   })

            max_ratio = ratios['Ratio'].max() - 0.1

    matched_name = ratios[ratios['Ratio'] >= max_ratio]

    x = record[position_column]
    #TODO
    #x= translator.translate(x, dest='en').text

    positions = employees[['Name', 'Position']].drop_duplicates().merge(matched_name, how='inner', on='Name')[['Name', 'Position', 'Ratio']]

    pos_ratios = pd.DataFrame({
        prefix + 'MatchedName': positions['Name'].tolist(),
        prefix + 'MatchedNameRatio': positions['Ratio'].tolist(),
        prefix + 'MatchedPosition': positions['Position'].tolist(),
        prefix + 'MatchedPositionRatio': [difflib.SequenceMatcher(None, x.lower() if x is not None else ' '
                                                                  , name.lower() if name is not None else ' ').ratio() for name in positions['Position'].tolist()]
    })
    pos_max_ratio = pos_ratios[prefix + 'MatchedPositionRatio'].max()
    matched_pos = pos_ratios[pos_ratios[prefix + 'MatchedPositionRatio'] >= pos_max_ratio]

    matched_pos = matched_pos[matched_pos[prefix + 'MatchedNameRatio'].max() == matched_pos[prefix + 'MatchedNameRatio']]

    return matched_pos

In [683]:
matched_employees = salaries.groupby(['CorrectedFinanceName', 'FinanceEnglishPosition']).apply(get_matched_record).reset_index()

ABIKE STRZEBINCZYK
0.2222222222222222
ABIKE STRZEBINCZYK
0.2222222222222222


ADAM AUKSEL
0.3
ADAM BASTRZYK
0.2727272727272727
ADAM KOWALCZYK
0.25


ADAM LASON
0.3157894736842105
ADAM LEPKOWSKI
0.2608695652173913
ADAM LUCZAK
0.2857142857142857
ADRIAN ANDRZEJEWSKI


0.21428571428571427
ADRIAN DOBROWOLSKI
0.2222222222222222
ADRIAN DOLNIAK
0.25


ADRIAN DUDACZYK
0.24
ADRIAN MACZUGA
0.25
ADRIAN MUCHA
0.2857142857142857
ADRIAN NOWAKOWSKI

...

WOJCIECH DOLISZNY
0.23076923076923078
WOJCIECH DUTKIEWICZ
0.21428571428571427
WOJCIECH DZIKOWICZ
0.2222222222222222
WOJCIECH JAKUBOWSKI
0.20689655172413793


WOJCIECH KACZMAREK
0.2222222222222222
WOJCIECH KROLIKOWSKI
0.2
WOJCIECH LUCIOW
0.20689655172413793


WOJCIECH MACHNIAK
0.23076923076923078
WOJCIECH MIODEK
0.25
WOJCIECH NOWAK
0.2608695652173913
WOJCIECH OSTROWSKI
0.21428571428571427


WOJCIECH SOLTYSIK
0.2222222222222222
WOJCIECH WYZGA
0.2608695652173913
YAROSLAV GONTAR
0.23076923076923078
ZBIGNIEW RUBINSKI
0.2222222222222222


ZUZANNA KOKOSZKA
0.24


In [684]:
matched_employees = matched_employees.drop(['level_2'], axis=1)

In [685]:
merged_salary = salaries.merge(matched_employees, how="outer")
merged_salary.shape


(706, 14)

In [686]:
merged_salary['Ratio'] = merged_salary['MatchedNameRatio'] * merged_salary['MatchedPositionRatio']

In [687]:
merged_salary[(~merged_salary['MatchedPosition'].isnull()) & (merged_salary['Ratio'] < 0.2)][['FinanceName', 'FinancePosition', 'MatchedPosition']].drop_duplicates()

,FinanceName,FinancePosition,MatchedPosition
1,ADAMSKA-WOŹNY ALEKSANDRA,Assistant Guild Master,Assistants Guild Manager
2,ANDRUSIECZKO ANDRZEJ,Junior Quality Engineer/ Młodszy Tester Oprogr...,Junior Quality Engineer
3,ANDRZEJEWSKI ADRIAN,Chapter Leader Service Delivery Manager,Chapter Leader Service Delivery Manager
4,ANTONETS IULIIA,ANALITYK BIZNESOWY,Quality Engineer
6,Badura ALEKSANDER,Senior Automation Quality Engineer/Starszy Tester,Automation Quality Engineer
10,BARTNIKIEWICZ AGNIESZKA,Personal Assistant,Attract Lead HR Business Partner
11,BARTOSIEWICZ ANNA,Finance Project Office Specialist,Finance Project Office Specialist
13,BEDNAREK AGATA,BUSINESS ANALYST / Analityk Biznesowy,Business Analyst
15,BIENKIEWICZ JOANNA,Quality Engineer,Quality Engineer
19,BŁAŻKOWSKI PIOTR,TEAM LEADER SENIOR QUALITY ENGINEER,Team Leader Senior Quality Engineer


In [688]:
employees = employees[['Name', 'Position', 'Tribe', 'Subtribe', 'Guild', 'Subguild', 'Office', 'Level', 'values',
                       'LeaveDate', 'EmploymentDate', 'Management', 'Date', 'Backoffice', 'LastWorkingDate', 'Sex']]

In [689]:
employees['Date'] = employees['Date'].apply(lambda x : x.date())
merged_salary['Date'] = pd.to_datetime(merged_salary['Date'])
merged_salary['Date'] = merged_salary['Date'].apply(lambda x : x.date())
merged_salary = merged_salary.merge(employees, how="outer", left_on=['MatchedName', 'Date'], right_on=['Name', 'Date'])
merged_salary.head()
salaries.shape[0], merged_salary.shape[0]
merged_salary['Date'].unique()
count = merged_salary.groupby(['Name', 'Date']).apply(lambda x: pd.DataFrame({'Count': [x.shape[0]]}))
count[count['Count'] > 1]
merged_salary.head()
merged_salary[(merged_salary['Name'] == 'Monika Grzech')]['Ratio'].head()
merged_salary['ContractType'].unique()


array(['Na czas nieokreślony', 'B2B', 'Na okres próbny',
       'Na czas określony', nan], dtype=object)

In [690]:
FIN_SERVER = "OBJPLDMFIN1"
FIN_DATABASE = "Cost"
FIN_SCHEMA = "dbo"
FIN_DB_CONNECTION = 'mssql+pyodbc://@' + FIN_SERVER + '/' + FIN_DATABASE + '?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes'
FIN_ENGINE = sqlalchemy.create_engine(FIN_DB_CONNECTION, pool_size=20, max_overflow=100)

In [691]:
contract_data = merged_salary


In [692]:
merged_salary.to_sql('Salaries_ContractValues_Tmp', FIN_ENGINE.connect(), if_exists='replace', index=False)